In [1]:
# We need to predict real disaster vs not real disaster

In [2]:
import logging


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [3]:
import pandas as pd
df = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
df.rename(columns={'text': 'text', 'target': 'labels'}, inplace=True)

# Drop unnecessary columns for simplicity, keeping only 'text' and 'labels'
df = df[['text', 'labels']]

print(df.head())

                                                text  labels
0  Our Deeds are the Reason of this #earthquake M...       1
1             Forest fire near La Ronge Sask. Canada       1
2  All residents asked to 'shelter in place' are ...       1
3  13,000 people receive #wildfires evacuation or...       1
4  Just got sent this photo from Ruby #Alaska as ...       1


In [5]:
from sklearn.model_selection import train_test_split

train_df, eval_df = train_test_split(df, test_size=0.10, random_state=42)

In [6]:
!pip -q install simpletransformers

In [7]:
from transformers import AutoTokenizer
from simpletransformers.classification import ClassificationModel
import torch

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased", truncation=True, use_cuda=torch.cuda.is_available())

model_args = {
    "sliding_window": True,
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "num_train_epochs": 8,
    "train_batch_size": 32,
    "tokenizer": tokenizer
}

model = ClassificationModel(
    "bert", "bert-base-uncased", args=model_args
)

2024-07-17 15:30:17.916181: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-17 15:30:17.916315: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-17 15:30:18.060331: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [8]:
import sklearn
model.train_model(train_df, eval_data=eval_df)

/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/6851 [00:00<?, ?it/s]

Epoch:   0%|          | 0/8 [00:00<?, ?it/s]

Running Epoch 1 of 8:   0%|          | 0/215 [00:00<?, ?it/s]

Running Epoch 2 of 8:   0%|          | 0/215 [00:00<?, ?it/s]

...

Running Epoch 5 of 8:   0%|          | 0/215 [00:00<?, ?it/s]

Running Epoch 6 of 8:   0%|          | 0/215 [00:00<?, ?it/s]

Running Epoch 7 of 8:   0%|          | 0/215 [00:00<?, ?it/s]

Running Epoch 8 of 8:   0%|          | 0/215 [00:00<?, ?it/s]

(1720, 0.18045736917573418)

In [9]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/762 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/8 [00:00<?, ?it/s]

In [10]:
print(result)

{'mcc': 0.6178012020131131, 'tp': 268, 'tn': 350, 'fp': 76, 'fn': 68, 'eval_loss': 1.0579272955656052}


In [11]:
df_test = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

In [12]:
# Assuming your model is already trained and loaded as `model`
predictions, raw_outputs = model.predict(df_test['text'].tolist())

/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/3263 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

In [13]:
results_df = pd.DataFrame({
    'id': df_test['id'],
    'target': predictions
})

print(results_df.head())

results_df.to_csv('/kaggle/working/submission.csv', index=False)

   id  target
0   0       1
1   2       1
2   3       1
3   9       1
4  11       1
